# 02 Linear Probe Segmentation

Train a simple linear head on top of frozen DINOv3 features.

In [1]:
import torch
import matplotlib.pyplot as plt
from PIL import Image
from transformers import AutoImageProcessor
from dinov3_lab.core.backbone import build_dinov3_hf
from dinov3_lab.tasks.segmentation.heads import LinearSegmentationHead

# 1. Setup
backbone = build_dinov3_hf()
head = LinearSegmentationHead(in_channels=1024, num_classes=150)
processor = AutoImageProcessor.from_pretrained("facebook/dinov3-vitl16-pretrain-lvd1689m")

# 2. Load Image
image_path = "../data/test_images/demo.jpg"
try:
    image = Image.open(image_path).convert("RGB")
except FileNotFoundError:
    print("Demo image not found. Using random noise.")
    image = Image.new('RGB', (448, 448), color = 'gray')

inputs = processor(images=image, return_tensors="pt")

# 3. Forward Pass
with torch.no_grad():
    out = backbone(inputs.pixel_values)
    
# Convert tokens to grid
grid = backbone.tokens_to_grid(out.patch_tokens, out.patch_hw)
print(f"Grid shape: {grid.shape}")

# 4. Head Prediction
logits = head(grid)
print(f"Logits shape: {logits.shape}")

# Upsample to image size
image_size = image.size[::-1] # (H, W)
logits_up = backbone.upsample_grid_to_image(logits, image_size)
print(f"Upsampled logits: {logits_up.shape}")

# Visualize argmax (untrained head, so random noise)
pred_mask = logits_up.argmax(dim=1)[0].cpu().numpy()

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title("Input")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(pred_mask)
plt.title("Prediction (Untrained)")
plt.axis("off")
plt.show()

2025-12-14 16:33:42.519070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765730022.538644   10256 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765730022.544682   10256 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-14 16:33:42.563492: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
`torch_dtype` is deprecated! Use `dtype` instead!
[W1214 16:33:46.880258542 NNPACK.cpp:61] Could not initialize NNPACK! Reaso

Grid shape: torch.Size([1, 768, 14, 14])


RuntimeError: Input type (c10::BFloat16) and bias type (float) should be the same